In [1]:
import requests
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
class ShopifyGraphQLClient:
    def __init__(self):
        self.endpoint = f"https://{'waveit-742'}.myshopify.com/admin/api/2025-10/graphql.json"
        self.headers = {
            'Content-Type': 'application/json',
            'X-Shopify-Access-Token': os.getenv("SHOPIFY_ACCESS_TOKEN")
        }
    
    def execute_query(self, query, variables=None):
        payload = {'query': query}
        if variables:
            payload['variables'] = variables
        
        response = requests.post(self.endpoint, json=payload, headers=self.headers)
        response.raise_for_status()
        return response.json()

In [3]:
client = ShopifyGraphQLClient()

In [4]:
query = """
query {
  shop {
    id
    name
    email
    billingAddress {
      id
      address1
    }
  }
  products(first:10) {
    nodes {
      id
      title
      description
    }
  }
}
"""

In [5]:
client.execute_query(query)

{'data': {'shop': {'id': 'gid://shopify/Shop/79647113434',
   'name': 'waveIt',
   'email': 'alapatipavanvk@gmail.com',
   'billingAddress': {'id': 'gid://shopify/ShopAddress/79647113434',
    'address1': ''}},
  'products': {'nodes': [{'id': 'gid://shopify/Product/9112596349146',
     'title': 'SoundWave Pro Bluetooth Earbuds',
     'description': 'Experience exceptional sound quality with SoundWave Pro Bluetooth Earbuds. Featuring noise cancellation and a comfortable fit, these earbuds are perfect for music enthusiasts on the go.'},
    {'id': 'gid://shopify/Product/9112596742362',
     'title': 'Echo Studio High-Fidelity Smart Speaker',
     'description': 'Experience premium sound with Echo Studio, curated for an immersive audio atmosphere and seamlessly compatible with Alexa.'},
    {'id': 'gid://shopify/Product/9112597037274',
     'title': 'EchoBlast Smart Home Speaker',
     'description': 'Experience exceptional sound clarity with the EchoBlast Smart Home Speaker, featuring vo

In [2]:
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333")

In [3]:
import time
import populate_store

RUNS = 50
KEYWORD = input()  # or any keyword you want

for i in range(RUNS):
    print(f"\n🚀 Run {i+1}/{RUNS}")

    products = populate_store.generate_5_products_with_gpt4(
        KEYWORD,
        n=1
    )

    for p in products:
        print("📦 Generated product:", p["title"])
        populate_store.create_shopify_product(p)

    time.sleep(6)



🚀 Run 1/50
🤖 Generating 1 distinct 'electronics' products...
📦 Generated product: AcousticZen H700 Noise-Cancelling Headphones
✅ Created: AcousticZen H700 Noise-Cancelling Headphones | ID: 9116248178906
   ⚠️ No Taxonomy ID generated by GPT. Skipping category assignment.

🚀 Run 2/50
🤖 Generating 1 distinct 'electronics' products...
📦 Generated product: EchoPlus Smart Speaker with Alexa
✅ Created: EchoPlus Smart Speaker with Alexa | ID: 9116249456858
   ⚠️ No Taxonomy ID generated by GPT. Skipping category assignment.

🚀 Run 3/50
🤖 Generating 1 distinct 'electronics' products...
📦 Generated product: HomeGuard 360 Smart Security Camera
✅ Created: HomeGuard 360 Smart Security Camera | ID: 9116250669274
   ⚠️ No Taxonomy ID generated by GPT. Skipping category assignment.

🚀 Run 4/50
🤖 Generating 1 distinct 'electronics' products...
📦 Generated product: Echo Guru Smart Home Assistant
✅ Created: Echo Guru Smart Home Assistant | ID: 9116251881690
   ⚠️ No Taxonomy ID generated by GPT. Skippi

In [11]:
import os
from qdrant_client import QdrantClient
from openai import OpenAI

qdrant_client = QdrantClient(
    url="http://localhost:6333", 
    
)

openai_client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

def search_products(query_text, limit=5):
    """
    Search for products semantically similar to the query_text.
    """
    
    # 2. Convert text to vector (Embedding)
    # CRITICAL: Use the SAME model here that you used to index your data.
    # Common models: "text-embedding-3-small", "text-embedding-ada-002"
    embedding_response = openai_client.embeddings.create(
        input=query_text,
        model="text-embedding-3-small" 
    )
    
    query_vector = embedding_response.data[0].embedding

    # 3. Search Qdrant
    search_results = qdrant_client.query_points(
        collection_name="shopify_products",
        query=query_vector,
        limit=limit,
        with_payload=True # Ensures you get the product title/price/description back
    )

    return search_results

# --- Example Usage ---
user_query = "I am looking for a hp laptops"
results = search_products(user_query)

print(f"Results for: '{user_query}'\n")
for hit in results:
    # 'hit.payload' contains the original JSON data (title, price, etc.)
    for result in hit[1]:

        product_name = result.payload.get('title', 'Unknown Product')
        score = result.score # Similarity score (higher is better)

        print(product_name, f"(Score: {score:.4f})")
        #print(results)
        

Results for: 'I am looking for a hp laptops'

HP Spectre x360 Convertible - Premium 2-in-1 Laptop (Score: 0.4675)
HP Spectre x360 Convertible - Premium 2-in-1 Laptop (Score: 0.4582)
HP Spectre x360 Convertible Laptop (Score: 0.4550)
HyperBlade Pro 17 Gaming Laptop (Score: 0.4042)
Predator Helios Gaming Laptop 2024 (Score: 0.4020)


In [5]:
import os
from qdrant_client import QdrantClient
from openai import OpenAI

qdrant_client = QdrantClient(url="http://localhost:6333")
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def search_products(query_text, limit=5):
    embedding_response = openai_client.embeddings.create(
        input=query_text,
        model="text-embedding-3-small"
    )
    query_vector = embedding_response.data[0].embedding

    return qdrant_client.query_points(
        collection_name="shopify_products",
        query=query_vector,
        limit=limit,
        with_payload=True,
        with_vectors=False
    )

# --- Example Usage ---
user_query = "I am looking for a keyboard"
results = search_products(user_query, limit=5)

print(f"\nResults for: '{user_query}'\n")

for i, point in enumerate(results.points, 1):
    payload = point.payload or {}
    title = payload.get("title", "Unknown Product")
    vendor = payload.get("vendor", "")
    price = payload.get("price", "")
    score = float(point.score)

    print(f"{i}. {title} | {vendor} | ${price} (Score: {score:.4f})")


Results for: 'I am looking for a keyboard'

1. Interactive Learning Tablet for Kids | BrainyTykes | $29.99 (Score: 0.2769)
2. PhantomGamer Pro X15 | TechGalaxy | $1499.99 (Score: 0.2724)
3. TechPro X7 Gaming Laptop | TechPro Electronics | $1499.99 (Score: 0.2667)
4. Predator Triton 500 SE | Acer | $2199.99 (Score: 0.2661)
5. Falcon Pro Gaming Laptop | Falcon Tech | $1499.99 (Score: 0.2645)
